# Joining tables

**Author:** 'Felipe Millacura'
    
**Date:** '07th February 2021'

## Learning Objectives

* Be familiar with the different types of table relationships
* Understand how keys create relationships
* Be able to interact with SQL databases in Jupyter notebooks
* Be able to identify the correct join to use
* Understand `INNER JOIN`, `LEFT JOIN` and `RIGHT JOIN`
* Be able to use all join types in Pandas

## Introduction

So far we have been extracting, manipulating and returning data from **single tables**. But we know from our discussion that **relationships** between tables are important. After all, we are learning about **relational databases** in general.

Some questions arise:

**"What sort of relationships can we have between tables?"**

**"Will we ever need to simultaneously access data from multiple tables to solve a problem?"**

**"If so, how do we write a single SQL query to do this?"**


This lesson will start by answering the first question via a short section of theory. 


### Table relationships

Two tables can be related to each other in three ways:

* One-to-one
* One-to-many
* Many-to-many

where 'one' and 'many' refer to records (i.e. rows) in each of the tables. So the phrase 'one-to-one' should be read in full as 'one row in table A is linked to one row in table B', and so on. We'll discuss these relationships one at a time with examples of their use, but first we need to talk about **foreign keys**!


## Foreign keys

Relationships between tables are set up by adding **foreign key columns**. We've already seen the `PRIMARY KEY` column of a table: this establishes a **unique identifier** (usually an `INT`) for each row.<br><br>

A `FOREIGN KEY` is a column in a table that contains values corresponding to the primary keys of **another** table

Recall the `zoo` example from the SQL lesson. We ended up with two tables: `animals` and `diets`. <br><br>

**animals**

| `id` | `name` | `age` | `species` | `diet_id` |
| --- | --- | --- | --- | --- |
| 1 | Leo | 12 | Lion | 2 |
| 2 | Tony | 8 | Tiger | 2 |
| 3 | Matilda | 6 | Cow | 1 |
| 4 | Bernice | 12 | Bear | 3 |

<br>

**diets**

| `id` | `diet_type` |
| --- | --- |
| 1 | herbivore |
| 2 | carnivore |
| 3 | omnivore |

The `id` columns of both tables are `PRIMARY KEY`s: these contain the unique identifiers for each row. The `diet_id` column of table `animals` is a `FOREIGN KEY`: it contains values corresponding to the `PRIMARY KEY`s of the `diets` table.

The three table relationships we mentioned above come down to **how** we insert foreign keys into tables, and what **constraints** we put upon them! 

## One-to-one

Consider a government tax database containing two tables: `citizens` and `tax_details`. We expect that **one** row in `citizens` will be linked to **one** row in `tax_details`, and vice-versa. So **each** citizen has **one** set of tax details, and **one** set of tax details will belong to **one** citizen. 
![](images/One-to-One.png)

How do we arrange foreign keys to achieve this one-to-one relationship? We put a foreign key in **one** of the tables and make it **unique**. Here's the table specification:



```sql
CREATE TABLE citizens (
  id INT PRIMARY KEY,
  --and other fields...
)

CREATE TABLE tax_details (
  id INT PRIMARY KEY,
  citizen_id INT UNIQUE NOT NULL REFERENCES citizens(id),
  --and other fields...
)
```

In this case we make the `citizen_id` foreign key `NOT NULL` as well, as it doesn't make much sense for this use case to have a tax details record that is not linked to a citizen.

The `REFERENCES` operator is what sets up the foreign key: we state that the `citizen_id` of the `tax_details` table will contains `id` values from the `citizens` table.

Here's a summary of the table design:

* each `tax_details` record **must** be linked to **one and only one** `citizens` record
* each `citizens` record is linked to **zero or one** `tax_details` record

Strictly speaking, this is a **one-to-zero-or-one** relationship. True one-to-one relationships are better implemented within a single table.

## One-to-many

Next, think about a database for a corporation with tables for `employees` and `teams` that the employees may belong to. The relationship we wish to model is that **one** employee must belong to **one** team, but a team may have **many** employees.

![](images/One-to-Many.png)

We implement this in a table design as



```sql
CREATE TABLE teams (
  id INT PRIMARY KEY,
  --and other fields...
)

CREATE TABLE employees (
  id INT PRIMARY KEY,
  team_id INT NOT NULL REFERENCES teams(id),
  --and other fields...
)
```

Notice we've removed the `UNIQUE` constraint here: we want to allow multiple employees to belong to a single team. 

## Many-to-many

Finally, imagine a project planning application supported by a database with tables for `employees` and `projects`. The relationship we wish to model is that an employee can be involved in **many** projects, and a project can involve many **employees**.

For a many-to-many relationship, we need to insert a third table, called a **join table** to hold the details of the relationship. Two foreign keys will be held in the join table, which will also have it's own primary key and may have other fields.

The join table in this case is called `involvements`, as it tracks the involvement of employees in projects. However, if there is no better name, the convention is just to name it *tableA_tableB*, where *tableA* and *tableB* are the names of the joined tables. So, in the present case, we might have called the join table `employees_projects`, but we'll stick with `involvements`.

![](images/Many-to-Many.png)


Let's see the table design!

```sql
CREATE TABLE employees (
  id INT PRIMARY KEY,
  --and other fields...
)

CREATE TABLE projects (
  id INT PRIMARY KEY,
  --and other fields...
)

CREATE TABLE involvements (
  id INT PRIMARY KEY,
  employee_id INT NOT NULL REFERENCES employees(id),
  project_id INT NOT NULL REFERENCES projects(id).
  --and other fields, potentially...
)
```

Let's summarise the design:

* each `employees` record can be linked to **zero, one or many** `involvements` records.
* each `involvements` record **must** be linked to **one** `employees` record.

and

* each `projects` record can be linked to **zero, one or many** `involvements` records.
* each `involvements` record **must** be linked to **one** `projects` record.

So you can see that we have replaced a single many-to-many relationship with **two** one-to-many relationships.

### Recap

* What is a foreign key?

A foreign key is a field in a table that contains primary key values of another table.

* What SQL keyword sets up a foreign key in a table?

`REFERENCES` (or `FOREIGN KEY`)

* What are the three types of table relationships we have discussed here?

  - One-to-one 
  - One-to-many
  - Many-to-many
  
* What is added to the database design to set up a many-to-many relationship?

Typically we add an extra **join table** that contains foreign keys linked to the primary keys of the two tables in the many-to-many relationship.




## How do we interact with an SQL database in Jupyter notebooks?

Using **IPython SQL** Magic extension. **Magic commands** are a set of convenient functions in Jupyter Notebooks that are designed to solve some of the common problems in standard data analysis. You can see all of the available magics with the help of `%lsmagic`.

In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

[IPython SQL](https://github.com/catherinedevlin/ipython-sql) magic extension makes it possible to write SQL queries directly into code cells as well as read the results straight into pandas DataFrames

##### First we need to install some packages

In [2]:
 !pip install ipython-sql

##### Loading the SQL module

In [3]:
%load_ext sql

The above magic command loads the `ipython-sql` extension. Again we'll use the `zoo` database to help you understand the concepts of this lesson, but this time we'll create an actual working `zoo` database in SQLite!


In [4]:
%sql  sqlite://

In [5]:
%%sql 

CREATE TABLE animals (
        id INT PRIMARY KEY,
        name VARCHAR(50),
        age INT,
        species VARCHAR(50),
        diet_id INT REFERENCES diets(id)
);

CREATE TABLE diets (
 id INT PRIMARY KEY,
 diet_type VARCHAR(50)
);

CREATE TABLE keepers (
        id INT PRIMARY KEY,
        name VARCHAR(50)
);

CREATE TABLE care_schedule (
    id INT PRIMARY KEY,
    animal_id INT NOT NULL REFERENCES animals(id),
    keeper_id INT NOT NULL REFERENCES keepers(id),
    day day_of_week NOT NULL
);

INSERT INTO animals (id, name, age, species, diet_id) VALUES (1, 'Leo', 12, 'Lion', 2);
INSERT INTO animals (id, name, age, species, diet_id) VALUES (2, 'Tony', 8, 'Tiger', 2);
INSERT INTO animals (id, name, age, species, diet_id) VALUES (3, 'Matilda', 6, 'Cow', 1);
INSERT INTO animals (id, name, age, species, diet_id) VALUES (4, 'Bernice', 12, 'Bear', 3);
INSERT INTO animals (id, name, age, species)          VALUES (5, 'Gerry', 1, 'Goldfish');
INSERT INTO animals (id, name, age, species, diet_id) VALUES (6, 'Zoe', 3, 'Zebra', 1);
INSERT INTO animals (id, name, age, species, diet_id) VALUES (7, 'Ernest', 4, 'Snake', 2);
INSERT INTO animals (id, name, age, species)          VALUES (8, 'Kim', 6, 'Kangaroo');

INSERT INTO diets (id, diet_type) VALUES (1, 'herbivore');
INSERT INTO diets (id, diet_type) VALUES (2, 'carnivore');
INSERT INTO diets (id, diet_type) VALUES (3, 'omnivore');
INSERT INTO diets (id, diet_type) VALUES (4, 'tofu');


INSERT INTO keepers (id, name) VALUES (1, 'Tony');
INSERT INTO keepers (id, name) VALUES (2, 'Yoshi');
INSERT INTO keepers (id, name) VALUES (3, 'Anne');
INSERT INTO keepers (id, name) VALUES (4, 'Jerome');


INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (1, 1, 2, 'Monday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (2, 1, 1, 'Tuesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (3, 1, 3, 'Wednesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (4, 1, 4, 'Thursday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (5, 1, 2, 'Friday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (6, 1, 1, 'Saturday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (7, 1, 4, 'Sunday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (8, 2, 3, 'Monday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (9, 2, 4, 'Tuesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (10, 2, 3, 'Wednesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (11, 2, 1, 'Thursday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (12, 2, 4, 'Friday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (13, 2, 2, 'Saturday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (14, 2, 3, 'Sunday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (15, 3, 1, 'Monday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (16, 3, 4, 'Tuesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (17, 3, 4, 'Wednesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (18, 3, 1, 'Thursday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (19, 3, 3, 'Friday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (20, 3, 4, 'Saturday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (21, 3, 2, 'Sunday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (22, 4, 4, 'Monday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (23, 4, 2, 'Tuesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (24, 4, 3, 'Wednesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (25, 4, 1, 'Thursday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (26, 4, 3, 'Friday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (27, 4, 4, 'Saturday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (28, 4, 2, 'Sunday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (29, 5, 3, 'Monday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (30, 5, 1, 'Tuesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (31, 5, 4, 'Wednesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (32, 5, 2, 'Thursday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (33, 5, 4, 'Friday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (34, 5, 3, 'Saturday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (35, 5, 1, 'Sunday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (36, 6, 1, 'Monday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (37, 6, 2, 'Tuesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (38, 6, 4, 'Wednesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (39, 6, 3, 'Thursday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (40, 6, 2, 'Friday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (41, 6, 4, 'Saturday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (42, 6, 1, 'Sunday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (43, 7, 2, 'Monday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (44, 7, 3, 'Tuesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (45, 7, 1, 'Wednesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (46, 7, 4, 'Thursday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (47, 7, 4, 'Friday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (48, 7, 3, 'Saturday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (49, 7, 2, 'Sunday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (50, 8, 1, 'Monday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (51, 8, 3, 'Tuesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (52, 8, 1, 'Wednesday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (53, 8, 4, 'Thursday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (54, 8, 3, 'Friday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (55, 8, 2, 'Saturday');
INSERT INTO care_schedule (id, animal_id, keeper_id, day) VALUES (56, 8, 2, 'Sunday');


 * sqlite://
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affe

[]

Let's query all records in the `animals`, `diets`, `keepers` and `care_schedule` tables to prove the code is working.

In [6]:
%sql SELECT * FROM animals

 * sqlite://
Done.


id,name,age,species,diet_id
1,Leo,12,Lion,2
2,Tony,8,Tiger,2
3,Matilda,6,Cow,1
4,Bernice,12,Bear,3
5,Gerry,1,Goldfish,None
6,Zoe,3,Zebra,1
7,Ernest,4,Snake,2
8,Kim,6,Kangaroo,None


In [7]:
%sql SELECT * FROM diets

 * sqlite://
Done.


id,diet_type
1,herbivore
2,carnivore
3,omnivore
4,tofu


In [8]:
%sql SELECT * FROM keepers

 * sqlite://
Done.


id,name
1,Tony
2,Yoshi
3,Anne
4,Jerome


In [9]:
%sql SELECT * FROM care_schedule

 * sqlite://
Done.


id,animal_id,keeper_id,day
1,1,2,Monday
2,1,1,Tuesday
3,1,3,Wednesday
4,1,4,Thursday
5,1,2,Friday
6,1,1,Saturday
7,1,4,Sunday
8,2,3,Monday
9,2,4,Tuesday
10,2,3,Wednesday



We have a few more animals in the zoo now, and one more diet type: `'tofu'`, which we guess will not prove very popular with the animals!

### `INNER JOIN`

How can we solve the following problem:

    "Get a list of all the animals that have diet plans together with the diet plans that they are on."

We use an `INNER JOIN`! Let's write the query and then describe what it does.

In [10]:
%%sql

SELECT animals.*, diets.*
           FROM animals INNER JOIN diets
           ON animals.diet_id = diets.id

 * sqlite://
Done.


id,name,age,species,diet_id,id_1,diet_type
1,Leo,12,Lion,2,2,carnivore
2,Tony,8,Tiger,2,2,carnivore
3,Matilda,6,Cow,1,1,herbivore
4,Bernice,12,Bear,3,3,omnivore
6,Zoe,3,Zebra,1,1,herbivore
7,Ernest,4,Snake,2,2,carnivore



* `animals INNER JOIN diets` tells SQL to join the `animals` table to the `diets` table.
* `ON animals.diet_id = diets.id` tells SQL **how to do the join**. This says 'take the rows from `animals` and the rows from `diets` and line up any rows where the `diet_id` in an `animals` row equals the `id` in a `diets` row'.
* `INNER JOIN` means 'return only those rows where there is such a match'.

You can see in the results above that the `diet_id` values from `animals` all match the `id` values from `diets`.

It might help to think of 'cutting' both tables up into separate records, and then 'glueing' records together into a joined result, so that primary key matched foreign key and creating duplicate records if needed to balance the numbers required. 

![](images/One_to_many_join.png)

We can control the columns returned:

In [11]:
%%sql 
SELECT animals.name, animals.species, diets.diet_type
           FROM animals INNER JOIN diets
           ON animals.diet_id = diets.id

 * sqlite://
Done.


name,species,diet_type
Leo,Lion,carnivore
Tony,Tiger,carnivore
Matilda,Cow,herbivore
Bernice,Bear,omnivore
Zoe,Zebra,herbivore
Ernest,Snake,carnivore


and we can also use **table aliases** to make queries more compact to write:

In [12]:
%%sql 
SELECT a.name, a.species, d.diet_type
           FROM animals AS a INNER JOIN diets AS d
           ON a.diet_id = d.id

 * sqlite://
Done.


name,species,diet_type
Leo,Lion,carnivore
Tony,Tiger,carnivore
Matilda,Cow,herbivore
Bernice,Bear,omnivore
Zoe,Zebra,herbivore
Ernest,Snake,carnivore


Previously we've seen `AS` being used in the `SELECT` clause to alias columns. Here we use it in the `FROM` clause to alias tables.

We can use all the other advanced query syntax we learned earlier. For example:

    "Find any known dietary requirements for animals over four years old."

In [13]:
%%sql SELECT a.id, a.name, a.species, a.age, d.diet_type
           FROM animals AS a INNER JOIN diets AS d
           ON a.diet_id = d.id
           WHERE a.age > 4

 * sqlite://
Done.


id,name,species,age,diet_type
1,Leo,Lion,12,carnivore
2,Tony,Tiger,8,carnivore
3,Matilda,Cow,6,herbivore
4,Bernice,Bear,12,omnivore


    "Breakdown the number of animals in the zoo by their diet types."

In [14]:
%%sql SELECT d.diet_type, COUNT(a.id)
           FROM animals AS a INNER JOIN diets AS d
           ON a.diet_id = d.id
           GROUP BY d.diet_type

 * sqlite://
Done.


diet_type,COUNT(a.id)
carnivore,3
herbivore,2
omnivore,1


### `LEFT JOIN` and `RIGHT JOIN`

So, `INNER JOIN` returns only those joined records where there is match in the condition we specify. Let's see this again


In [15]:
%%sql 
SELECT a.*, d.*
           FROM animals AS a INNER JOIN diets AS d
           ON a.diet_id = d.id

 * sqlite://
Done.


id,name,age,species,diet_id,id_1,diet_type
1,Leo,12,Lion,2,2,carnivore
2,Tony,8,Tiger,2,2,carnivore
3,Matilda,6,Cow,1,1,herbivore
4,Bernice,12,Bear,3,3,omnivore
6,Zoe,3,Zebra,1,1,herbivore
7,Ernest,4,Snake,2,2,carnivore


There's no sign of Gerry the Goldfish, or Kim the Kangaroo, as these animals have `NULL`s for `diet_id`. Ditto, tofu is missing from the table, as no animal has a `diet_id` of 4 (i.e. no animal prefers tofu).

But what if we want **all** of the records from **one** of the tables, regardless of whether there is a match in the condition. This is where `LEFT JOIN` and `RIGHT JOIN` come in! See the following problem.

    "Return the details of ALL animals in the zoo, together with their dietary requirements if they have any."

In [16]:
%%sql 
SELECT a.*, d.*
           FROM animals AS a LEFT JOIN diets AS d
           ON a.diet_id = d.id

 * sqlite://
Done.


id,name,age,species,diet_id,id_1,diet_type
1,Leo,12,Lion,2,2,carnivore
2,Tony,8,Tiger,2,2,carnivore
3,Matilda,6,Cow,1,1,herbivore
4,Bernice,12,Bear,3,3,omnivore
5,Gerry,1,Goldfish,None,None,None
6,Zoe,3,Zebra,1,1,herbivore
7,Ernest,4,Snake,2,2,carnivore
8,Kim,6,Kangaroo,None,None,None


Now we see all the animals, including Gerry and Kim, although they have no dietary requirements.

So `LEFT JOIN` says 'keep all the records in the left table, adding in any matching records from the right table'. 'Left' here indicates the table to the left of the `LEFT JOIN` operator, i.e. `animals`.

### `FULL OUTER JOIN`


What happens if we want all results from both tables whether they have matches or not. `FULL OUTER JOIN` combines the results of both left and right outer joins and returns all (matched or unmatched) rows from the tables on both sides of the join.

Let's take a look at the results of a full outer join in our animals and diets example:


In [17]:
%%sql

SELECT a.*, d.*
           FROM animals AS a FULL OUTER JOIN diets AS d
           ON a.diet_id = d.id

 * sqlite://
(sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: SELECT a.*, d.* FROM animals AS a FULL OUTER JOIN diets AS d
           ON a.diet_id = d.id]
(Background on this error at: http://sqlalche.me/e/13/e3q8)



We can see that for entries where there is a join they are returned, but so do Gerry the Goldfish and Kim the Kangaroo, who have no match in the diet table, and the diet tofu, even though there is no matching animal for this. 

This picture representation of the different joins can be useful to refer to:

![](images/joins_summary.png)


### Joins in many-to-many relationships

Finally, let's see how to use joins to extract data from tables linked by a many-to-many relationship. Remember we have an extra **join table** in a many-to-many table.

In the `zoo` database, we have a many-to-many relationship between animals and keepers, i.e.<br><br>

<center>'Each animal is cared for by many keepers, and each keeper cares for many animals'.</center><br>

The join table is called `care_schedule`. Each record in `care_schedule` tells us which keeper is looking after which animal, on which day.<br>

![](images/Many-to-Many-Zoo.png)


In order to join `animals` to `keepers` we have to go in two 'hops': first, a join from `animals` to `care_schedule`; and second, a join from `care_schedule` to `keepers`. 

Let's write the query and then describe it step by step. Don't be put off, this looks complicated, but think of building it up bit by bit...

In [18]:
%%sql

SELECT a.name AS animal_name, cs.day, k.name AS keeper_name
             FROM (animals AS a INNER JOIN care_schedule AS cs
             ON a.id = cs.animal_id)
             INNER JOIN keepers AS k
             ON cs.keeper_id = k.id
             ORDER BY a.name, cs.day

 * sqlite://
Done.


animal_name,day,keeper_name
Bernice,Friday,Anne
Bernice,Monday,Jerome
Bernice,Saturday,Jerome
Bernice,Sunday,Yoshi
Bernice,Thursday,Tony
Bernice,Tuesday,Yoshi
Bernice,Wednesday,Anne
Ernest,Friday,Jerome
Ernest,Monday,Yoshi
Ernest,Saturday,Anne


You see we have two `JOIN` operators. Here's one way to visualise this:<br>

![](images/Multi-step-joins.png)

Think of the output table from `animals` to `care_schedule` being **further joined** to `keepers`, i.e. <br><br>

<center>(`animals INNER JOIN care_schedule`) `INNER JOIN keepers`</center><br>

The `INNER JOIN` in parentheses is executed first, and the output from it then `INNER JOIN`ed to `keepers`.

We use an `INNER JOIN`! Let's write the query and then describe what it does.

### What about using an existing database?

We can connect to any database which is supported by [SQLAlchemy](https://www.sqlalchemy.org/). Here we will connect to a SQLite database making use of the [Soccer database](https://d17h27t6h515a5.cloudfront.net/topher/2017/November/5a0a4cad_database/database.sqlite) which can be chosen as part of your third project. Available completly at [Kaggle](https://www.kaggle.com/hugomathien/soccer)

In [19]:
%sql sqlite:////Users/fmill/Documents/Udacity/part_3/session_9/database.sqlite

Now we can make standard SQL queries to our connected database. Let's select all records from the `Country` table

In [20]:
%sql SELECT name FROM Country

   sqlite://
 * sqlite:////Users/fmill/Documents/Udacity/part_3/session_9/database.sqlite
Done.


name
Belgium
England
France
Germany
Italy
Netherlands
Poland
Portugal
Scotland
Spain


We can filter using the `WHERE` clause

In [21]:
%sql SELECT * FROM Country WHERE name = 'Spain';

   sqlite://
 * sqlite:////Users/fmill/Documents/Udacity/part_3/session_9/database.sqlite
Done.


id,name
21518,Spain


The results can also be converted to a `pandas` dataframe, as follows:  

In [22]:
result = %sql SELECT * FROM Match WHERE country_id = 21518;  

df = result.DataFrame()

   sqlite://
 * sqlite:////Users/fmill/Documents/Udacity/part_3/session_9/database.sqlite
Done.


Now you can analyse as usual

In [23]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,3040.0,2.303750e+04,877.716735,21518.00,22277.75,23037.50,23797.250,24557.0
country_id,3040.0,2.151800e+04,0.000000,21518.00,21518.00,21518.00,21518.000,21518.0
league_id,3040.0,2.151800e+04,0.000000,21518.00,21518.00,21518.00,21518.000,21518.0
stage,3040.0,1.950000e+01,10.967660,1.00,10.00,19.50,29.000,38.0
match_api_id,3040.0,1.214688e+06,495063.577676,530023.00,827888.00,1155674.50,1574151.750,2030537.0
...,...,...,...,...,...,...,...,...
GBD,1893.0,3.888722e+00,1.300759,2.75,3.25,3.40,3.750,11.0
GBA,1893.0,4.693698e+00,4.002445,1.12,2.70,3.50,5.000,34.0
BSH,1899.0,2.559963e+00,1.937599,1.04,1.70,2.05,2.500,17.0
BSD,1899.0,3.870084e+00,1.284813,2.75,3.25,3.40,3.750,13.0


In [24]:
df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,21518,21518,21518,2008/2009,1,2008-08-30 00:00:00,530023,10267,8661,3,...,5.00,1.65,3.40,4.50,1.75,3.30,5.00,1.67,3.40,5.00
1,21519,21518,21518,2008/2009,1,2008-08-31 00:00:00,530084,8371,10205,1,...,2.60,2.90,3.20,2.25,2.75,3.20,2.50,2.80,3.25,2.30
2,21520,21518,21518,2008/2009,1,2008-08-31 00:00:00,530085,9783,8633,2,...,1.90,3.25,3.25,2.00,3.50,3.20,2.10,3.25,3.25,2.10
3,21521,21518,21518,2008/2009,1,2008-08-31 00:00:00,530086,8388,8634,1,...,1.57,7.00,3.75,1.45,6.50,3.75,1.53,5.50,3.75,1.53
4,21522,21518,21518,2008/2009,1,2008-08-31 00:00:00,530087,8696,8302,1,...,2.30,2.70,3.10,2.40,2.75,3.25,2.45,2.70,3.25,2.38


Let's import some other information. What about the `Team` table

In [25]:
result = %sql SELECT * FROM Team;  

df_teams = result.DataFrame()

   sqlite://
 * sqlite:////Users/fmill/Documents/Udacity/part_3/session_9/database.sqlite
Done.


In [26]:
df_teams.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


### Joins and Merges in Pandas

As you learnt about all the theory behind different join types. Now we're going to practice doing them in Python using the `pandas` package

The `pandas` package contains functions to perform all of the joins you can have in SQL:


`merge()` for combining data on common columns or indices

`.join()` for combining data on a key column or an index

`concat()` for combining DataFrames across rows or columns


In [27]:
import pandas as pd

You can join DataFrames by concatenating `df1` and `df2` on the common column (or key) `id`. To do so, pass the names of the DataFrames and an additional argument `on` as the name of the common column, here  `team_api_id`, to the `merge()` function:

In [28]:
df_merged = pd.merge(df, df_teams, left_on='home_team_api_id', right_on='team_api_id')

df_merged

,id_x,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBD,GBA,BSH,BSD,BSA,id_y,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,21518,21518,21518,2008/2009,1,2008-08-30 00:00:00,530023,10267,8661,3,...,3.30,5.00,1.67,3.40,5.0,43035,10267,461.0,Valencia CF,VAL
1,21539,21518,21518,2008/2009,11,2008-11-15 00:00:00,530244,10267,9869,2,...,4.25,8.50,1.36,4.33,7.5,43035,10267,461.0,Valencia CF,VAL
2,21562,21518,21518,2008/2009,13,2008-11-30 00:00:00,530327,10267,8603,3,...,3.40,4.60,1.83,3.30,4.2,43035,10267,461.0,Valencia CF,VAL
3,21586,21518,21518,2008/2009,15,2008-12-13 00:00:00,530351,10267,8558,2,...,3.75,6.00,1.53,4.00,6.0,43035,10267,461.0,Valencia CF,VAL
4,21605,21518,21518,2008/2009,17,2009-01-03 00:00:00,530370,10267,9906,3,...,3.30,3.25,2.20,3.30,3.2,43035,10267,461.0,Valencia CF,VAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,24485,21518,21518,2015/2016,37,2016-05-08 00:00:00,2030525,8306,8315,0,...,NaN,NaN,NaN,NaN,NaN,48358,8306,472.0,UD Las Palmas,LAS
3036,24498,21518,21518,2015/2016,4,2015-09-20 00:00:00,2030112,8306,8370,0,...,NaN,NaN,NaN,NaN,NaN,48358,8306,472.0,UD Las Palmas,LAS
3037,24508,21518,21518,2015/2016,5,2015-09-23 00:00:00,2030122,8306,8302,2,...,NaN,NaN,NaN,NaN,NaN,48358,8306,472.0,UD Las Palmas,LAS
3038,24529,21518,21518,2015/2016,7,2015-10-03 00:00:00,2030143,8306,8372,0,...,NaN,NaN,NaN,NaN,NaN,48358,8306,472.0,UD Las Palmas,LAS


### Join DataFrames

The logic behind these joins is very much the same that you have in SQL when you join tables. You can find the `pandas` specifications [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)

#### Inner Join

Inner join will return all records in two tables that match on some condition.


In [29]:
df_inner = pd.merge(df, df_teams, left_on='home_team_api_id', right_on='team_api_id', how='inner')

df_inner

,id_x,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBD,GBA,BSH,BSD,BSA,id_y,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,21518,21518,21518,2008/2009,1,2008-08-30 00:00:00,530023,10267,8661,3,...,3.30,5.00,1.67,3.40,5.0,43035,10267,461.0,Valencia CF,VAL
1,21539,21518,21518,2008/2009,11,2008-11-15 00:00:00,530244,10267,9869,2,...,4.25,8.50,1.36,4.33,7.5,43035,10267,461.0,Valencia CF,VAL
2,21562,21518,21518,2008/2009,13,2008-11-30 00:00:00,530327,10267,8603,3,...,3.40,4.60,1.83,3.30,4.2,43035,10267,461.0,Valencia CF,VAL
3,21586,21518,21518,2008/2009,15,2008-12-13 00:00:00,530351,10267,8558,2,...,3.75,6.00,1.53,4.00,6.0,43035,10267,461.0,Valencia CF,VAL
4,21605,21518,21518,2008/2009,17,2009-01-03 00:00:00,530370,10267,9906,3,...,3.30,3.25,2.20,3.30,3.2,43035,10267,461.0,Valencia CF,VAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,24485,21518,21518,2015/2016,37,2016-05-08 00:00:00,2030525,8306,8315,0,...,NaN,NaN,NaN,NaN,NaN,48358,8306,472.0,UD Las Palmas,LAS
3036,24498,21518,21518,2015/2016,4,2015-09-20 00:00:00,2030112,8306,8370,0,...,NaN,NaN,NaN,NaN,NaN,48358,8306,472.0,UD Las Palmas,LAS
3037,24508,21518,21518,2015/2016,5,2015-09-23 00:00:00,2030122,8306,8302,2,...,NaN,NaN,NaN,NaN,NaN,48358,8306,472.0,UD Las Palmas,LAS
3038,24529,21518,21518,2015/2016,7,2015-10-03 00:00:00,2030143,8306,8372,0,...,NaN,NaN,NaN,NaN,NaN,48358,8306,472.0,UD Las Palmas,LAS


### Left join

Left join will return all of the records in the left table, regardless of whether they have a matching record in the right table, but it will also return any matching records in the right table.

In [30]:
df_left = pd.merge(df, df_teams, left_on='home_team_api_id', right_on='team_api_id', how='left')

df_left

,id_x,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBD,GBA,BSH,BSD,BSA,id_y,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,21518,21518,21518,2008/2009,1,2008-08-30 00:00:00,530023,10267,8661,3,...,3.30,5.00,1.67,3.40,5.00,43035,10267,461.0,Valencia CF,VAL
1,21519,21518,21518,2008/2009,1,2008-08-31 00:00:00,530084,8371,10205,1,...,3.20,2.50,2.80,3.25,2.30,43037,8371,479.0,CA Osasuna,OSA
2,21520,21518,21518,2008/2009,1,2008-08-31 00:00:00,530085,9783,8633,2,...,3.20,2.10,3.25,3.25,2.10,43039,9783,242.0,RC Deportivo de La Coruña,COR
3,21521,21518,21518,2008/2009,1,2008-08-31 00:00:00,530086,8388,8634,1,...,3.75,1.53,5.50,3.75,1.53,43041,8388,477.0,CD Numancia,NUM
4,21522,21518,21518,2008/2009,1,2008-08-31 00:00:00,530087,8696,8302,1,...,3.25,2.45,2.70,3.25,2.38,43043,8696,456.0,Racing Santander,SAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,24553,21518,21518,2015/2016,9,2015-10-25 00:00:00,2030167,9906,10267,2,...,NaN,NaN,NaN,NaN,NaN,43053,9906,240.0,Atlético Madrid,AMA
3036,24554,21518,21518,2015/2016,9,2015-10-24 00:00:00,2030168,9864,9783,2,...,NaN,NaN,NaN,NaN,NaN,43054,9864,573.0,Málaga CF,MAL
3037,24555,21518,21518,2015/2016,9,2015-10-26 00:00:00,2030169,8315,9869,3,...,NaN,NaN,NaN,NaN,NaN,43051,8315,448.0,Athletic Club de Bilbao,BIL
3038,24556,21518,21518,2015/2016,9,2015-10-24 00:00:00,2030170,7878,8603,1,...,NaN,NaN,NaN,NaN,NaN,45330,7878,110832.0,Granada CF,GRA


### Right Join

Right join is a mirror version of the left join - it returns all records in the right table, along with any matching records in the left table.

In [31]:
df_right = pd.merge(df, df_teams, left_on='home_team_api_id', right_on='team_api_id', how='right')

df_right

,id_x,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBD,GBA,BSH,BSD,BSA,id_y,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,9987,673.0,KRC Genk,GEN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,9993,675.0,Beerschot AC,BAC
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4,9994,2007.0,Sporting Lokeren,LOK
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,49479,10190,898.0,FC St. Gallen,GAL
3302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,49837,10191,1715.0,FC Thun,THU
3303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50201,9777,324.0,Servette FC,SER
3304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50204,7730,1862.0,FC Lausanne-Sports,LAU


#### Full Outer Join

Outer join combines the results of both the left and the right outer joins. The joined DataFrame will contain all records from both the DataFrames and fill in NaNs for missing matches on either side. 

In [32]:
df_outer = pd.merge(df, df_teams, left_on='home_team_api_id', right_on='team_api_id', how='outer')

df_outer

,id_x,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,GBD,GBA,BSH,BSD,BSA,id_y,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,21518.0,21518.0,21518.0,2008/2009,1.0,2008-08-30 00:00:00,530023.0,10267.0,8661.0,3.0,...,3.30,5.00,1.67,3.40,5.0,43035,10267,461.0,Valencia CF,VAL
1,21539.0,21518.0,21518.0,2008/2009,11.0,2008-11-15 00:00:00,530244.0,10267.0,9869.0,2.0,...,4.25,8.50,1.36,4.33,7.5,43035,10267,461.0,Valencia CF,VAL
2,21562.0,21518.0,21518.0,2008/2009,13.0,2008-11-30 00:00:00,530327.0,10267.0,8603.0,3.0,...,3.40,4.60,1.83,3.30,4.2,43035,10267,461.0,Valencia CF,VAL
3,21586.0,21518.0,21518.0,2008/2009,15.0,2008-12-13 00:00:00,530351.0,10267.0,8558.0,2.0,...,3.75,6.00,1.53,4.00,6.0,43035,10267,461.0,Valencia CF,VAL
4,21605.0,21518.0,21518.0,2008/2009,17.0,2009-01-03 00:00:00,530370.0,10267.0,9906.0,3.0,...,3.30,3.25,2.20,3.30,3.2,43035,10267,461.0,Valencia CF,VAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,49479,10190,898.0,FC St. Gallen,GAL
3302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,49837,10191,1715.0,FC Thun,THU
3303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50201,9777,324.0,Servette FC,SER
3304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50204,7730,1862.0,FC Lausanne-Sports,LAU


That's all for Today! 

![](https://preview.redd.it/yku64jkyz9121.jpg?width=960&crop=smart&auto=webp&s=5e7667e19de00d4026698eadd0383aca00ade77b)